In [26]:
import numpy as np
import librosa
from scipy.io import wavfile
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
from scipy.signal import filtfilt


In [2]:
%config Completer.use_jedi = False


In [53]:
##copied
def get_time_smoothed_representation(
    spectral, samplerate, hop_length, time_constant_s=0.001
):
    t_frames = time_constant_s * samplerate / float(hop_length)
    # By default, this solves the equation for b:
    #   b**2  + (1 - b) / t_frames  - 2 = 0
    # which approximates the full-width half-max of the
    # squared frequency response of the IIR low-pass filt
    b = (np.sqrt(1 + 4 * t_frames ** 2) - 1) / (2 * t_frames ** 2)
    return filtfilt([b], [1, b - 1], spectral, axis=-1, padtype=None)

def sigmoid(x,shift,mult):
    
    return 1/(1+np.exp(-(x+shift)*mult))



In [70]:
def remove_nonstationary_noise(audio,
                               sr,
                               n_fft=2048,
                               win_length=2048,
                               hop_length=512,
                               prop_decrease=1,
                               shift_sigmoid=2,
                               mult_sigmoid=10,
                               n_grad_freq=2,
                               n_grad_time=4,
                               time_constant=0.001):
    
    audio_stft = librosa.stft(audio,n_fft=n_fft,win_length=win_length,hop_length=hop_length)
    audio_stft_abs = np.abs(audio_stft)
    
    audio_time_smoothed = get_time_smoothed_representation(audio_stft,sr,hop_length,time_constant)
    
    audio_mask = (audio_stft_abs - audio_time_smoothed)/audio_time_smoothed
    audio_mask = sigmoid(audio_mask,shift_sigmoid,mult_sigmoid)
    
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter/np.sum(smoothing_filter)
    
    audio_mask = fftconvolve(audio_mask,smoothing_filter,mode='same')
    audio_mask = audio_mask * prop_decrease + np.ones(audio_mask.shape) * (1-prop_decrease)
    
    denoised_audio_stft = audio_stft * audio_mask
    
    return librosa.istft(denoised_audio_stft,win_length=win_length,hop_length=hop_length)
    
    

In [71]:
def get_noise(datalen,min_freq,max_freq,rate):

    freqs = np.abs(np.fft.fftfreq(n=datalen,d=1/rate))
    samples = np.zeros(datalen)
    samples[np.logical_and(freqs>=min_freq,freqs<=max_freq)]=1

    samples = np.array(samples,dtype='complex')
    Np = (len(samples)-1)//2
    phases = np.random.rand(Np) * 2 * np.pi
    e_iphases = np.cos(phases) + 1j * np.sin(phases)
    samples[1:Np+1] *= e_iphases
    samples[Np+1:-1] = np.conj(samples[1:Np+1])
    noise = (np.fft.ifft(samples).real)

    return noise

In [72]:
rate,audio = wavfile.read("/Users/shahules/Myprojects/Audio-Exercise/test_set_noise.wav")
#rate,noise = wavfile.read("/Users/shahules/Myprojects/Noise-removal-audio/notebooks/noise.wav")
audio = audio/2**15
Audio(audio,rate=rate)

In [73]:
noise = get_noise(len(audio),3000,10000,rate)*10
Audio(noise,rate=rate)

In [74]:
correpted_audio = audio+noise

In [75]:
Audio(data=correpted_audio,rate=rate)

In [76]:
filted = remove_nonstationary_noise(audio=audio,sr=rate)

In [77]:
Audio(data=filted,rate=rate)